In [1]:
# Sequential A/B testing (low-traffic friendly) — simple, reproducible notebook snippet
# Idea: we "peek" only at pre-planned checkpoints and use an adjusted threshold
# (here: Bonferroni-style alpha split). This is illustrative, not production-grade.

import numpy as np
from scipy.stats import norm

rng = np.random.default_rng(7)

# --- Setup (edit these) ---
p_a_true = 0.10           # baseline conversion rate (A)
p_b_true = 0.15           # treatment conversion rate (B) (make this closer to p_a_true to see slower decisions)

checkpoints = [50, 100, 150, 200, 300, 400]  # per-variant sample sizes where we allow a "peek"
alpha = 0.05                                # overall two-sided Type I error target (roughly controlled here)

# Split alpha across planned looks (simple + conservative)
K = len(checkpoints)
alpha_per_look = alpha / K
z_crit = norm.ppf(1 - alpha_per_look / 2)  # two-sided

# --- Simulate sequential arrivals in blocks ---
a_conversions = 0
b_conversions = 0
n_a = 0
n_b = 0

print(f"Planned peeks: {checkpoints}")
print(f"Per-look alpha: {alpha_per_look:.4f}  ->  |z| >= {z_crit:.2f} triggers stop\n")

for target_n in checkpoints:
    # add new users since the last checkpoint
    add = target_n - n_a
    a_block = rng.binomial(1, p_a_true, size=add)
    b_block = rng.binomial(1, p_b_true, size=add)

    a_conversions += int(a_block.sum())
    b_conversions += int(b_block.sum())
    n_a = target_n
    n_b = target_n

    # --- standard two-sample z-test for difference in proportions (A vs B) ---
    p_a_hat = a_conversions / n_a
    p_b_hat = b_conversions / n_b
    p_pool = (a_conversions + b_conversions) / (n_a + n_b)

    se = np.sqrt(p_pool * (1 - p_pool) * (1 / n_a + 1 / n_b))
    z = (p_b_hat - p_a_hat) / se if se > 0 else 0.0
    p_value_nominal = 2 * (1 - norm.cdf(abs(z)))  # nominal (not sequential-adjusted)

    lift_hat = (p_b_hat / p_a_hat - 1) if p_a_hat > 0 else np.inf

    print(
        f"n/variant={n_a:>3} | A={p_a_hat:.3f} ({a_conversions:>3}/{n_a}) "
        f"B={p_b_hat:.3f} ({b_conversions:>3}/{n_b}) | "
        f"lift≈{lift_hat:+.1%} | z={z:+.2f} | nominal p={p_value_nominal:.3f}"
    )

    # --- Sequential stopping rule (planned + symmetric: win or harm) ---
    if z >= z_crit:
        print("\nSTOP : evidence B > A at a planned checkpoint (with per-look adjusted threshold).")
        break
    if z <= -z_crit:
        print("\nSTOP : evidence B < A (harm) at a planned checkpoint (with per-look adjusted threshold).")
        break
else:
    print("\nNO STOP: reached final checkpoint without crossing a boundary.")

Planned peeks: [50, 100, 150, 200, 300, 400]
Per-look alpha: 0.0083  ->  |z| >= 2.64 triggers stop

n/variant= 50 | A=0.060 (  3/50) B=0.160 (  8/50) | lift≈+166.7% | z=+1.60 | nominal p=0.110
n/variant=100 | A=0.110 ( 11/100) B=0.150 ( 15/100) | lift≈+36.4% | z=+0.84 | nominal p=0.400
n/variant=150 | A=0.087 ( 13/150) B=0.133 ( 20/150) | lift≈+53.8% | z=+1.29 | nominal p=0.196
n/variant=200 | A=0.100 ( 20/200) B=0.140 ( 28/200) | lift≈+40.0% | z=+1.23 | nominal p=0.218
n/variant=300 | A=0.090 ( 27/300) B=0.127 ( 38/300) | lift≈+40.7% | z=+1.44 | nominal p=0.148
n/variant=400 | A=0.083 ( 33/400) B=0.125 ( 50/400) | lift≈+51.5% | z=+1.97 | nominal p=0.049

NO STOP: reached final checkpoint without crossing a boundary.
